#SCRAP HISTORICAL DATA


In [109]:
from datetime import datetime
from datetime import date
from bs4 import BeautifulSoup

from dateutil.relativedelta import relativedelta 
import requests
import time
import re

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style
%matplotlib notebook

In [115]:
def get_data(start_month, start_year, end_month, end_year, symbol, get_dividend = False):
  t = datetime(1970, 1, 1)
  tstart = datetime(start_year, start_month, 1)
  tend = datetime(end_year, end_month+1, 1)

  #num_days = (tend-tstart).days

  num_months = (tend.year - tstart.year) * 12 + (tend.month - tstart.month)

  A = []
  B = []
  C = []

  for month in range(num_months):
    tmonth = (tstart + relativedelta(months= +1))
    tickstart = (tstart - t).total_seconds()
    tickend = (tmonth - t).total_seconds()

    url = "https://finance.yahoo.com/quote/"+symbol+"/history?period1="+str(int(tickstart))+"&period2="+str(int(tickend))+"&interval=1d&filter=history&frequency=1d"

    response =  requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    price = soup.find_all('tr', {"C($tertiaryColor) Fz(xs) Ta(end)"})[0]
    iter = 0
    for i in range(len(soup.find_all('td'))):
      
      price = soup.find_all('td')[i].text

      if "*Close price adjusted for splits.**Adjusted close price adjusted for both dividends and splits." in price:
        break
      
      A.append(price)

      if "Dividend" not in price:
        if iter%7 == 6:
          B.append(A)
          A = []
        iter = iter+1

      if "Dividend" in price:
        C.append(A)
        A = []
        iter = iter+6

    tstart = tmonth

  Dividend = pd.DataFrame(C , columns=['Date', 'Dividend'])
  Dividend['Date'] = pd.to_datetime(Dividend['Date'])
  Dividend.sort_values(by=['Date'], inplace=True, ascending=True)
  Dividend.set_index('Date', inplace = True)

  df = pd.DataFrame(B, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])
  df['Date']=pd.to_datetime(df['Date'])
  df.sort_values(by=['Date'], inplace=True, ascending=True)
  df.set_index('Date', inplace = True)

  for data in [df]:
    for col in data.columns:
      data[col] = data[col].apply(lambda x: re.sub(r'[^0-9.]+','',x))

  if get_dividend == True:
    return df, Dividend
  else:
    return df
  


In [119]:
START_MONTH = 1        #1-12
START_YEAR = 2018      #yyyy

END_MONTH = 6
END_YEAR = 2020

SYMBOL = 'INRJPY=X'

stock_data= get_data(start_month = START_MONTH, start_year = START_YEAR, end_month = END_MONTH, end_year = END_YEAR, symbol = SYMBOL)

In [ ]:
stock_data

#Scrap Every second Data

In [ ]:
def get_minute_price(time_interval_sec, symbol):
  price_graph = []
  start_time = time.time()
  while True:
    url = 'https://finance.yahoo.com/quote/'+symbol
    response =  requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    price = soup.find_all('div', {'class': 'My(6px) Pos(r) smartphone_Mt(6px)'})[0].find('span', {'class' : 'Trsdu(0.3s) Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(b)'}).text
    price_graph.append(price)

    with open('MyFile.txt', 'a+') as file:
      file.write(price)
      file.write('\n')

    time.sleep(time_interval_sec - ((time.time() - start_time) % time_interval_sec))
    present_time = time.time()
    print (price)
  file.close() 

In [ ]:
get_minute_price(time_interval_sec = 1, symbol = "^NSEI")

##Plot Animator if required